# 生成器

### 生成器函数 

对于<font color= red>普通函数</font>，只要调用了，那么<font color= red>不把里面的代码执行完毕誓不罢休</font>。而函数内部的 return xxx，就是调用之后的返回值。


对于<font color= red>生成器函数</font>，调用的时候并没有动，只是返回一个生成器对象，然后需要每次调用一次 _ _ next _ _ ，才往前走一步。通过不断地驱动生成器，最终将里面的代码执行完毕，然后将设置了返回值的 StopIteration 抛出来。


生成器函数的特点是，其内部会出现yield关键字。我们也<font color= red>可以把生成器看成是可以暂停的函数</font>，其中的 yield 就类似于 return，只不过可以有多个 yield。当执行到一个 yield 时，将值返回、同时暂停在此处。然后当调用 _ _ next _ _ 驱动时，从暂停的地方继续执行，直到找到下一个 yield。如果找不到下一个 yield，就会抛出 StopIteration 异常。


- yield 关键字
- _ _ next _ _ 方法 ： 驱动生成器执行
- send 方法  ： 驱动生成器执行 + 传递一个值
- throw 方法 ： 驱动生成器执行 + 传递一个异常
- close 方法 ： 关闭生成器
- StopIteration 异常、GeneratorExit 异常、RuntimeError 异常
- yeild from 关键字 ：后面必须跟一个可迭代对象，然后每次返回可迭代对象的一个值；作为委托生成器
- 生成器表达式


### 生成器的作用
生成器对象一般用于处理循环结构，应用得当的话可以<font color= red>极大优化内存使用率</font>。


- 比如：我们读取一个大文件。使用普通函数，直接将里面的内容全部读取出来，返回了一个列表。如果文件非常大，那么内存的开销可想而知。于是我们可以通过yield关键字，将普通函数变成一个生成器函数，返回了一个生成器对象，可以使用 for 循环遍历。

In [1]:
def read_file(file):
    return open(file, encoding="utf-8").readlines()

print(read_file("假装是大文件.txt"))

['自强！自强！学海何洋洋！谁欤操钥发其藏？\n', '鹭江深且长，致吾知于无央。\n', '吁嗟乎！南方之强！吁嗟乎！南方之强！\n', '自强！自强！人生何茫茫！谁欤普渡驾慈航？\n', '鹭江深且长，充吾爱于无疆。\n', '吁嗟乎！南方之强！吁嗟乎！南方之强！']


In [3]:
def read_file(file):
    with open(file, encoding="utf-8") as f:
        for line in f:
            yield line

data = read_file("假装是大文件.txt")
print(data)  # 这里返回了一个生成器对象，我们可以使用 for 循环遍历

print("------------")

for line in data:
    print(line, end="")  # 文件每一行自带换行符, 所以这里的 print 就不用换行符了

<generator object read_file at 0x000002095DA74BA0>
------------
自强！自强！学海何洋洋！谁欤操钥发其藏？
鹭江深且长，致吾知于无央。
吁嗟乎！南方之强！吁嗟乎！南方之强！
自强！自强！人生何茫茫！谁欤普渡驾慈航？
鹭江深且长，充吾爱于无疆。
吁嗟乎！南方之强！吁嗟乎！南方之强！

###  _ _ next _ _ 方法
调用生成器函数时，会创建一个生成器，但是里面的代码并没有执行，调用 _ _ next _ _ 方法时才会执行。当遇到 yield，会将生成器暂停、并返回yield后面的值。当再调用 _ _ next _ _ 已经找不到下一个 yield 时，生成器会抛出StopIteration，并将返回值设置在里面。

In [4]:
def gen():
    yield 123
    yield 456
    yield 789
    return "result"

g = gen() # 调用生成器函数时，会创建一个生成器，但是里面的代码并没有执行

print(g.__next__())  # 123 ，调用__next__方法时才会执行，当遇到 yield，会将生成器暂停、并返回yield后面的值

print(g.__next__())  # 456 ，再次执行__next__，生成器恢复执行，并在下一个yield处暂停

print(g.__next__())  # 789

try:
    g.__next__() # 再调用 __next__ 时，找不到下一个 yield ，生成器会抛出 StopIteration，并将返回值设置在里面
except StopIteration as e:
    print(f"返回值：{e.value}")  # result

123
456
789
返回值：result


### send 方法
send方法与 _ _ next _ _ 方法一样，也可以<font color= red>驱动生成器执行</font>，其不同之处在于<font color= red>send方法可以向生成器传递一个参数</font>。

In [7]:
def gen():
    res1 = yield "yield 1"
    print(f"res = {res1}") # 格式化字符串,加f后可以在字符串里面使用用花括号括起来的变量和表达式
    res2 = yield "yield 2"
    print(f"res = {res2}") # 格式化字符串,加f后可以在字符串里面使用用花括号括起来的变量和表达式
    res3 = yield "yield 3"
    return res3

g = gen()

print(g.__next__()) # 此时程序在第一个 yield 处暂停
print("------------")

# 调用 g.send(val) 依旧可以驱动生成器执行
# 同时还可以传递一个值，交给第一个 yield 左边的 res1
# 然后在第二个 yield处暂停
print(g.send("嘿嘿")) 
print("------------")

print(g.__next__())  # __next__没有传递参数的功能，因此打印 res2 得到 None
print("------------")

try:
    g.send("蛤蛤")  # 把“蛤蛤”传递给 res3
except StopIteration as e:  #  没有下一个yield，生成器抛出StopIteration，并把return返回值设置在里面
    print(f"返回值：{e.value}")  # 这里输出的 e 的值就是赋值后的 res3 的值“蛤蛤” 

yield 1
------------
res = 嘿嘿
yield 2
------------
res = None
yield 3
------------
返回值：蛤蛤


刚创建生成器的时候，里面的代码还没有执行。第一次驱动生成器的执行，叫做<font color= red>生成器的预激</font>。<font color= red>在生成器还没有被预激时，我们调用 send，里面只能传递一个 None</font>，也就是 g.send(None),或者调用 g._ _ next _ _()，否则报错。

传值是为了赋给 yield 左边的变量，这就意味着生成器必须至少被驱动一次、在某个 yield 处停下来才可以。而未被预激的生成器，它里面的代码未被执行，所以第一次驱动的时候只能传递一个 None 进去。

In [11]:
def gen():
    res1 = yield 123
    res2 = yield 456
    return "result"

g = gen()
g.send([])

TypeError: can't send non-None value to a just-started generator

In [12]:
try:
    g.send([])
except TypeError as e:
    print(e)

can't send non-None value to a just-started generator


### throw方法
throw 方法的作用和前两者类似，也是<font color= red>驱动生成器执行，并在下一个 yield 处暂停</font>。但它在调用的时候，可以<font color= red>传递一个异常进去</font>。

In [21]:
def gen():
    try:
        yield 123
    except ValueError as e:
        print(e)
    yield 456
    return "result"

g = gen()
print(g.__next__())
print("------------")

# 向生成器传递一个异常
# 如果当前生成器的暂停位置处无法捕获传递的异常，那么会将异常抛出来
# 如果能够捕获，那么会驱动生成器执行，并在下一个 yield 处暂停
# 然后返回 yield 后面的值
# 当前生成器在 yield 123 处暂停，而它所在位置能够捕获异常
# 所以不会报错，结果就是 456 会赋值给 val
print(g.throw(ValueError("抛出异常")))
print("------------")

try:
    g.__next__()
except StopIteration as e:
    print(f"返回值：{e.value}")

123
------------
抛出异常
456
------------
返回值：result


### close方法
<font color= red>关闭</font>生成器

In [1]:
def gen():
    yield 123
    yield 456
    return "result"

g = gen()
print(g.__next__())
g.close()
try:
    g.__next__() # 再次调用 __next__，会抛出 StopIteration
except StopIteration as e:
    print(e.value)  # None

123
None


### GeneratorExit 异常
如果我们<font color= red>删除或关闭</font>一个生成器，那么会往里面扔一个 GeneratorExit 进去。

还有一个需要注意的点，就是在捕获 GeneratorExit 之后，不可以再执行 yield，否则会抛出 RuntimeError（但不会终止程序）。

In [2]:
def gen():
    try:
        yield 123
    except GeneratorExit as e:
        print("生成器被删除了")
        #yield 456

g = gen()
g.__next__()

g.close()
print("------------")
#del g

生成器被删除了
------------


In [46]:
def gen():
    try:
        yield 123
    except GeneratorExit:
        print("生成器被删除")
        yield 456

g = gen()
g.__next__()
del g
print("抛出 RuntimeError，但不影响程序执行")

Exception ignored in: <generator object gen at 0x000002095DBBA0B0>
Traceback (most recent call last):
  File "C:\Users\lenovo\AppData\Local\Temp/ipykernel_13988/1569352348.py", line 10, in <module>
RuntimeError: generator ignored GeneratorExit


生成器被删除
抛出 RuntimeError，但不影响程序执行


In [7]:
# 使用 del 删除生成器，不捕获异常的话，不会出现报错
def gen():
    yield 123

g = gen()
g.__next__()
del g

In [41]:
# 关闭或删除生成器时向生成器扔一个 GeneratorExit进去，使用throw方法能达到类似的效果
def gen():
    try:
        yield 123
    except GeneratorExit as e:
        print("生成器被删除了")    
        yield 456

g = gen()
g.__next__()
g.throw(GeneratorExit)

生成器被删除了


456

In [44]:
# 但是使用throw如果不对异常进行捕获的话，异常会被抛出
def gen():
    yield 123

g = gen()
g.__next__()
g.throw(GeneratorExit)

GeneratorExit: 

<font color= red>生成器为什么要提供这一个机制呢？</font>直接删就完了，干嘛还要往生成器内部丢一个异常呢？答案是为了<font color= red>资源的清理和释放</font>。



在Python还未提供原生协程、或者 asyncio 还尚未流行起来的时候，很多开源的协程框架都是基于生成器实现的协程。而创建连接的逻辑，一般都会写在 yield 后面。但是<font color= red>这些连接在不用的时候，要不要进行释放呢？答案是肯定的。当我们将生成器删除的时候，就能够自动对连接进行释放了</font>。

还有一个需要注意的点，就是在捕获 GeneratorExit 之后，不可以再执行 yield，否则会抛出 RuntimeError（但不会终止程序）。

In [1]:
def _create_connection():
    # 一些逻辑
    try: 
        yield conn
    except GeneratorExit:
        conn.close()
    # 一些逻辑

### yield from
yield from 后面必须跟一个<font color= red>可迭代对象</font>(否则报错)，然后每次返回可迭代对象的一个值，其它表现和 yield 类似。

yield 对后面的值没有要求，直接将其返回。

In [1]:
def gen():
    yield from [1, 2, 3]
    return "result"

g = gen()
print(g.__next__())  # 1
print(g.__next__())  # 2
print(g.__next__())  # 3
try:
    g.__next__()
except StopIteration as e:
    print(e.value)  # result

1
2
3
result


In [11]:
# 编写一个函数，将列表扁平化
def flatten(lst):
    for item in lst:
        (yield from flatten(item)) \
            if isinstance(item, list) else (yield item)

lst = [1, [[[[3, 3], 5]]], [[[[[[[[[[[[[6]]]]], 8]]], "aaa"]]]], 250]]
print(list(flatten(lst)))  # [1, 3, 3, 5, 6, 8, 'aaa', 250]

[1, 3, 3, 5, 6, 8, 'aaa', 250]


### yield from 作为委托生成器
委托生成器会在委托生成器<font color= red>调用方</font>和<font color= red>子生成器</font>之间建立一个双向通道。

当需求比较复杂时，将生成器内部的部分操作委托给另一个生成器是有必要的。

In [4]:
def gen():  # gen() 子生成器 
    yield 123
    yield 456
    return "result"

def middle():  # middle() 是委托生成器
    res = yield from gen()
    print(f"接收到子生成器的返回值: {res}")
    #return "委托生成器"
    yield 789

g = middle()  # g 是调用方

# 而 yield from 会在调用方和子生成器之间建立一个双向通道，两者是可以互通的
# g.send、g.throw 都会直接传递给子生成器
print(g.__next__())  # 123
print(g.__next__())  # 456
print("------------")

# 如果再调用一次 __next__ 会有什么后果呢？按照之前的理解，应该会抛出 StopIteration
try:
    print(g.__next__())
except StopIteration as e:
    print(e.value)  # 委托生成器

123
456
------------
接收到子生成器的返回值: result
789


在<font color= red>第三次调用 _ _ next _ _ 的时候，确实抛了异常，但是委托生成器收到了子生成器的返回值</font>。也就是说，委托生成器在调用方和子生成器之间建立了双向通道，两者是直接通信的，但是当子生成器出现 StopIteration 时，委托生成器还要负责兜底。<font color= red>委托生成器会将子生成器抛出的 StopIteration 里面的 value 取出来，然后赋值给左侧的变量 res，并在自己内部继续寻找 yield</font>。


换句话说，当子生成器 return 之后，委托生成器会拿到返回值，并将子生成器抛出的异常给捕获掉。但是还没完，因为还要找到下一个 yield，那么从哪里找呢？显然是委托生成器的内部寻找，于是接下来就变成了调用方和委托生成器之间的通信。<font color= red>如果在委托生成器内部能找到下一个 yield，那么会将值返回给调用方。如果找不到，那么就重新构造一个 StopIteration，将异常抛出去</font>。此时异常的 value 属性，就是委托生成器的返回值。

In [12]:
# 大部分情况下，我们并不关注委托生成器的返回值，我们更关注的是子生成器。于是可以换种写法：
#  只输出子生成器的返回值
def gen():
    yield 123
    return "result"

def middle():
    yield (yield from gen())

g = middle()
print(g.__next__())  # 123
print(g.__next__())  # result

123
result


### 生成器表达式

In [ ]:
from typing import Generator

g = (x for x in range(10))
print(isinstance(g, Generator))  # True
print(g)  # <generator object <genexpr> at 0x...>

print(g.__next__())  # 0
print(g.__next__())  # 1

In [16]:
# 如果表达式是在一个函数里面，那么生成器表达式周围的小括号可以省略掉。
import random

d = [random.randint(1, 10) for _ in range(100)]
# 我们想统计里面大于 5 的元素的总和
# 下面的做法都是可以的
print(
    sum((x for x in d if x > 5)),
    sum(x for x in d if x > 5)
)

388 388


### 生成器表达式陷阱
生成器表达式还存在一些陷阱，一不小心就踩进去。至于是什么陷阱呢？——<font color= red>使用生成器表达式创建生成器的时候，in 后面的变量就已经确定好了，但其它的变量则不会</font>。

In [17]:
g = (叐叒叓叕叚叜叝 for x in [1, 2, 3]) 
#  不会报错，尽管 for 前面那一坨我们没有定义，但不要紧，因为生成器是惰性执行的。
#  生成器只有在执行的时候，才会去确定变量 ‘叐叒叓叕叚叜叝’ 究竟指向谁

In [20]:
g = (叐叒叓叕叚叜叝 for x in [1, 2, 3])
g.__next__() #  报错 NameError

NameError: name '叐叒叓叕叚叜叝' is not defined

In [19]:
g = (x for x in lst1) #  #  报错 NameError

NameError: name 'lst1' is not defined

In [21]:
# in 后面的 lst 不会被修改
lst = [1, 2, 3]
g = (x for x in lst)
lst = [4, 5, 6]
print(tuple(g))  # (1, 2, 3)

(1, 2, 3)


In [22]:
#  in 前面的 i 会被更新
i = 1
g = (x + i for x in [1, 2, 3])
i = 10
print(tuple(g))  # (11, 12, 13)

(11, 12, 13)


In [23]:
# g 被新的生成器引用，i更新
g = (x for x in [1, 2, 3, 4])
for i in [1, 10]:
    g = (x + i for x in g)

print(tuple(g))    

# 初始的 g，可以看成是 (1, 2, 3, 4)，因为 in 后面是啥，在创建生成器的时候就确定了；
# 第一次循环之后，g 就相当于 (1+i, 2+i, 3+i, 4+i)，此时 i=1,；
# 第二次循环之后，g 就相当于 (1+i+i, 2+i+i, 3+i+i, 4+i+i)，此时 i=10。

(21, 22, 23, 24)


## 小结
从 Python 的角度梳理了一遍生成器的相关知识。
- 关键字：yield; yield from; return
- 方法：_ _ next _ _; send; throw
- 异常：StopIteration; GenerateExit; RuntimeError
- 委托生成器：使用 yeild from 实现
- 生成器表达式：有陷阱